In [1]:
from imports import *

In [2]:
df = pd.read_excel('output/scrape_miami_vape_2024-07-22_15-20-52.xlsx')

In [3]:
df.head()

,shop,rating,total_ratings,type,address,website,number,latitude,longitude
0,Vape & Smoke Shop - Biscayne,4.8,"(1,006)",Vaporizer store,"2895 Biscayne Blvd, Miami, FL 33137, United St...",http://www.vapensmokeshop.com/portfolio-view/b...,+1 786-703-7692,25.804018,-80.191709
1,Fuego Life Vape Shop,4.9,(68),Vaporizer store,"401 Biscayne Blvd T5585, Miami, FL 33132, Unit...",http://www.fuegolifevape.com/,+1 786-788-5660,25.804094,-80.230334
2,Caeli Smoke & Vape,4.3,(70),Tobacco shop,"16 SW 8th St, Miami, FL 33130, United States",NaN,+1 786-830-6683,25.782817,-80.230106
3,Lou Square Vape Shop & Smoke Shop,NaN,NaN,Vaporizer store,"941 Brickell Ave, Miami, FL 33131, United States",http://lousquare.com/,+1 305-548-1240,25.782892,-80.230106
4,VAPOP VAPE SHOP & SMOKE SHOP,5.0,(1),Vaporizer store,"921 Brickell Ave, Miami, FL 33131, United States",https://vapopvending.com/,+1 786-835-9149,25.772395,-80.223272


In [4]:
search_keyword=[]
for _, row in df.iterrows():
    # Construct the formatted string
    formatted_str = (
        f"{row['shop']}, {row['address']} "
    )
    
    # Replace "nan" with an empty string
    formatted_str = formatted_str.replace("nan", "").strip()
    
    search_keyword.append(formatted_str)
    

In [5]:
search_keyword

['Vape & Smoke Shop - Biscayne, 2895 Biscayne Blvd, Miami, FL 33137, United States',
 'Fuego Life Vape Shop, 401 Biscayne Blvd T5585, Miami, FL 33132, United States',
 'Caeli Smoke & Vape, 16 SW 8th St, Miami, FL 33130, United States',
 'Lou Square Vape Shop & Smoke Shop, 941 Brickell Ave, Miami, FL 33131, United States',
 'VAPOP VAPE SHOP & SMOKE SHOP, 921 Brickell Ave, Miami, FL 33131, United States',
 'VAPOP VAPE SHOP & SMOKE SHOP, 501 N Miami Ave, Miami, FL 33136, United States',
 'Lou Square, 999 Brickell Ave # 410, Miami, FL 33131, United States',
 'Brickell Smoke Shop, 13 SW 7th St, Miami, FL 33130, United States',
 'THC Vape Oil & E Juice Miami, 30 SW 1st St suite 400, Miami, FL 33130, United States',
 'Wake n Vape, 461 NE 30th Terrace #9, Miami, FL 33137, United States',
 'district smoke shop, 30 NE 1st Ave, Miami, FL 33132, United States',
 'The Hot Corner Smoke Shop, 49 NE 2nd Ave, Miami, FL 33132, United States',
 'VAPOP VAPE SHOP AT PEPTIOS PLAZA, 901 Brickell Plaza, Miami

In [7]:
chromedriver_path = "C:/Users/miraj/OneDrive/Desktop/projectMapsData-main/chromedriver.exe"
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service)

driver.get("https://www.google.com/maps/@32.8191173,-97.3554595,10z?hl=en&entry=ttu")
sleep(1)

def searchplace():
        
    for placee in search_keyword:


        # Locate the search box using its id
        place = driver.find_element(By.ID, "searchboxinput")
        locc = placee
        place.clear()
        sleep(2)
        
        place.send_keys(locc)
        sleep(2)  # Added sleep to ensure the input is fully typed
        # Locate the search button using its id
        submit = driver.find_element(By.ID, "searchbox-searchbutton")
        submit.click()

        sleep(4)


        try:
            shop = driver.find_element(By.CLASS_NAME, "hfpxzc")
            shop.click()
            sleep(2)

        except:
            pass


        revieew_nav = driver.find_element(By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div:nth-child(3) > div > div > button:nth-child(2)')
        revieew_nav.click()
        sleep(3)

        sort = driver.find_element(By.CSS_SELECTOR, 'button[aria-label="Sort reviews"]')
        sort.click()
        sleep(3)

        new= driver.find_element(By.CSS_SELECTOR, '#action-menu > div:nth-child(2)')
        new.click() 
        sleep(4)


    # Scroll to load more content

        scrollable_div = driver.find_element(By.CSS_SELECTOR, "#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.XiKgde" )
        last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)

        scroll_count = 1
        max_scrolls = 5

        while scroll_count < max_scrolls:
            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
            sleep(2)
            new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
            if new_height == last_height:
                break
            last_height = new_height
            scroll_count += 1


        review_boxs = driver.find_elements(By.CLASS_NAME, "jJc9Ad")

        # Initialize lists to store names, times, and user comments
        name_ = []
        time_ = []
        user_comments = []
        Rating_Stars = []  

        for i in review_boxs:
            # Extract and store the name
            name = i.find_element(By.CLASS_NAME, "d4r55")
            name_.append(name.text)
            
            # Extract and store the time ago
            time_ago = i.find_element(By.CLASS_NAME, "rsqaWe")
            time_.append(time_ago.text)

            ratings = i.find_element(By.CLASS_NAME, "kvMYJc")
            aria_label_value = ratings.get_attribute("aria-label")
            Rating_Stars.append(aria_label_value)
            # Extract and clean the review text
            
            li_review = i.text.split('\n')[9:]  # Skip the first 9 lines
            li_review = [line for line in li_review if line not in ['\ue8dc', '\ue80d']]
            
            if li_review:
                li_review = li_review[:-2]  # Remove the last two items if the list is not empty
            else:
                li_review.append("BLANK")  # Add "BLANK" if the review is empty

            # To remove unnecessary words - Translated by Google
            li_review=[i.split('Translated by Google')[0].strip() if "Translated by Google" in i else i for i in li_review]
            
            # Join the cleaned review list into a single string
            user_comment = ' '.join(li_review)
            user_comments.append(user_comment)


        dict={
            
            "Shop_Name":locc.split(",")[0],
            'Name': name_,
            "Time_Of_Comment": time_,
            "Rating_Stars": Rating_Stars,
            "Review_Comment": user_comments
        }
        
        df = pd.DataFrame(dict)

        timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

        filename = f'output_reviews/{locc.split(",")[0]}_reviews_{timestamp}.xlsx'
    
        df.to_excel(filename, index=False)

    

searchplace()



KeyboardInterrupt



In [ ]:
chromedriver_path = "C:/Users/miraj/OneDrive/Desktop/projectMapsData-main/chromedriver.exe"
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service)

driver.get("https://www.google.com/maps/@32.8191173,-97.3554595,10z?hl=en&entry=ttu")
sleep(1)

def searchplace(placee):

     # Locate and clear the search box, then search
        place = driver.find_element(By.ID, "searchboxinput")
        place.clear()
        sleep(2)
        place.send_keys(placee)
        sleep(2)
        submit = driver.find_element(By.ID, "searchbox-searchbutton")
        submit.click()
        sleep(4)

        try:
            shop = driver.find_element(By.CLASS_NAME, "hfpxzc")
            shop.click()
            sleep(2)
        except:
            pass

        try:
            # Navigate to the reviews section
            revieew_nav = driver.find_element(By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div:nth-child(3) > div > div > button:nth-child(2)')
            revieew_nav.click()
            sleep(3)
        except:
            pass

        try:
            sort = driver.find_element(By.CSS_SELECTOR, 'button[aria-label="Sort reviews"]')
            sort.click()
            sleep(3)
        except:
            pass
        
        new = driver.find_element(By.CSS_SELECTOR, '#action-menu > div:nth-child(2)')
        new.click()
        sleep(4)

        # Scroll to load more content
        scrollable_div = driver.find_element(By.CSS_SELECTOR, "#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.XiKgde")
        last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
        
        scroll_count = 1
        max_scrolls = 5
        
        while scroll_count < max_scrolls:
            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
            sleep(2)
            new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
            if new_height == last_height:
                break
            last_height = new_height
            scroll_count += 1

        # Extract review data
        review_boxs = driver.find_elements(By.CLASS_NAME, "jJc9Ad")
        name_ = []
        time_ = []
        user_comments = []
        Rating_Stars = []

        for i in review_boxs:
            name = i.find_element(By.CLASS_NAME, "d4r55")
            name_.append(name.text)
            
            time_ago = i.find_element(By.CLASS_NAME, "rsqaWe")
            time_.append(time_ago.text)

            ratings = i.find_element(By.CLASS_NAME, "kvMYJc")
            aria_label_value = ratings.get_attribute("aria-label")
            Rating_Stars.append(aria_label_value)
            
            li_review = i.text.split('\n')[9:]
            li_review = [line for line in li_review if line not in ['\ue8dc', '\ue80d']]
            
            if li_review:
                li_review = li_review[:-2]
            else:
                li_review.append("BLANK")

            li_review = [i.split('Translated by Google')[0].strip() if "Translated by Google" in i else i for i in li_review]
            user_comment = ' '.join(li_review)
            user_comments.append(user_comment)

        # Save to Excel
        dict = {
            "Shop_Name": placee.split(",")[0],
            'Name': name_,
            "Time_Of_Comment": time_,
            "Rating_Stars": Rating_Stars,
            "Review_Comment": user_comments
        }
        
        df = pd.DataFrame(dict)
        timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
        filename = f'output_reviews/{placee.split(",")[0]}_reviews_{timestamp}.xlsx'
        df.to_excel(filename, index=False)

        

searchplace()

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from time import sleep
import pandas as pd
from datetime import datetime
import os
from concurrent.futures import ThreadPoolExecutor
import threading
import keyboard  # Install this package with 'pip install keyboard'

chromedriver_path = "C:/Users/miraj/OneDrive/Desktop/projectMapsData-main/chromedriver.exe"

# Global list to keep track of browser instances
browsers = []

def process_place(placee):
    global browsers
    # Initialize the Chrome driver
    service = Service(chromedriver_path)
    driver = webdriver.Chrome(service=service)
    browsers.append(driver)  # Add browser instance to the list
    
    try:
        # Open Google Maps
        driver.get("https://www.google.com/maps/@32.8191173,-97.3554595,10z?hl=en&entry=ttu")
        sleep(1)

        # Locate and clear the search box, then search
        place = driver.find_element(By.ID, "searchboxinput")
        place.clear()
        sleep(2)
        place.send_keys(placee)
        sleep(2)
        submit = driver.find_element(By.ID, "searchbox-searchbutton")
        submit.click()
        sleep(4)

        try:
            shop = driver.find_element(By.CLASS_NAME, "hfpxzc")
            shop.click()
            sleep(2)
        except:
            pass

        try:
            # Navigate to the reviews section
            revieew_nav = driver.find_element(By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div:nth-child(3) > div > div > button:nth-child(2)')
            revieew_nav.click()
            sleep(3)
        except:
            pass

        try:
            sort = driver.find_element(By.CSS_SELECTOR, 'button[aria-label="Sort reviews"]')
            sort.click()
            sleep(3)
        except:
            pass
        
        new = driver.find_element(By.CSS_SELECTOR, '#action-menu > div:nth-child(2)')
        new.click()
        sleep(4)

        # Scroll to load more content
        scrollable_div = driver.find_element(By.CSS_SELECTOR, "#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.XiKgde")
        last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
        
        scroll_count = 1
        max_scrolls = 5
        
        while scroll_count < max_scrolls:
            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
            sleep(2)
            new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
            if new_height == last_height:
                break
            last_height = new_height
            scroll_count += 1

        # Extract review data
        review_boxs = driver.find_elements(By.CLASS_NAME, "jJc9Ad")
        name_ = []
        time_ = []
        user_comments = []
        Rating_Stars = []

        for i in review_boxs:
            name = i.find_element(By.CLASS_NAME, "d4r55")
            name_.append(name.text)
            
            time_ago = i.find_element(By.CLASS_NAME, "rsqaWe")
            time_.append(time_ago.text)

            ratings = i.find_element(By.CLASS_NAME, "kvMYJc")
            aria_label_value = ratings.get_attribute("aria-label")
            Rating_Stars.append(aria_label_value)
            
            li_review = i.text.split('\n')[9:]
            li_review = [line for line in li_review if line not in ['\ue8dc', '\ue80d']]
            
            if li_review:
                li_review = li_review[:-2]
            else:
                li_review.append("BLANK")

            li_review = [i.split('Translated by Google')[0].strip() if "Translated by Google" in i else i for i in li_review]
            user_comment = ' '.join(li_review)
            user_comments.append(user_comment)

        # Save to Excel
        dict = {
            "Shop_Name": placee.split(",")[0],
            'Name': name_,
            "Time_Of_Comment": time_,
            "Rating_Stars": Rating_Stars,
            "Review_Comment": user_comments
        }
        
        df = pd.DataFrame(dict)
        timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
        filename = f'output_reviews/{placee.split(",")[0]}_reviews_{timestamp}.xlsx'
        df.to_excel(filename, index=False)
        
    finally:
        # Close the browser
        driver.quit()
        browsers.remove(driver)  # Remove browser instance from the list

def searchplace(search_keywords):
    with ThreadPoolExecutor(max_workers=5) as executor:
        executor.map(process_place, search_keywords)

def key_listener():
    while True:
        if keyboard.is_pressed('q'):  # Check if 'q' is pressed
            print("Stopping all browsers...")
            for driver in browsers:
                driver.quit()
            break

# Start the key listener in a separate thread
listener_thread = threading.Thread(target=key_listener)
listener_thread.start()

# Example usage

searchplace(search_keyword)


Stopping all browsers...


In [44]:
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.common.by import By
# from time import sleep
# import pandas as pd
# from concurrent.futures import ThreadPoolExecutor

# chromedriver_path = "C:/Users/miraj/OneDrive/Desktop/projectMapsData-main/chromedriver.exe"
# service = Service(chromedriver_path)
# driver = webdriver.Chrome(service=service)

# def searchplace(place):
#     driver.get("https://www.google.com/maps/@32.8191173,-97.3554595,10z?hl=en&entry=ttu")
#     sleep(1)

#     # Locate the search box using its id
#     search_box = driver.find_element(By.ID, "searchboxinput")
#     search_box.send_keys(place)
#     sleep(2)  # Added sleep to ensure the input is fully typed

#     # Locate the search button using its id
#     submit = driver.find_element(By.ID, "searchbox-searchbutton")
#     submit.click()
#     sleep(4)

#     try:
#         shop = driver.find_element(By.CLASS_NAME, "hfpxzc")
#         shop.click()
#         sleep(2)
#     except:
#         return

#     review_nav = driver.find_element(By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div:nth-child(3) > div > div > button:nth-child(2)')
#     review_nav.click()
#     sleep(3)

#     sort = driver.find_element(By.CSS_SELECTOR, 'button[aria-label="Sort reviews"]')
#     sort.click()
#     sleep(3)

#     new = driver.find_element(By.CSS_SELECTOR, '#action-menu > div:nth-child(2)')
#     new.click()
#     sleep(4)

#     # Scroll to load more content
#     scrollable_div = driver.find_element(By.CSS_SELECTOR, "#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.XiKgde")
#     last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)

#     scroll_count = 1
#     max_scrolls = 5

#     while scroll_count < max_scrolls:
#         driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
#         sleep(2)
#         new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
#         if new_height == last_height:
#             break
#         last_height = new_height
#         scroll_count += 1

#     review_boxes = driver.find_elements(By.CLASS_NAME, "jJc9Ad")

#     # Initialize lists to store names, times, and user comments
#     name_ = []
#     time_ = []
#     user_comments = []
#     Rating_Stars = []

#     for i in review_boxes:
#         # Extract and store the name
#         name = i.find_element(By.CLASS_NAME, "d4r55")
#         name_.append(name.text)
        
#         # Extract and store the time ago
#         time_ago = i.find_element(By.CLASS_NAME, "rsqaWe")
#         time_.append(time_ago.text)

#         ratings = i.find_element(By.CLASS_NAME, "kvMYJc")
#         aria_label_value = ratings.get_attribute("aria-label")
#         Rating_Stars.append(aria_label_value)
        
#         # Extract and clean the review text
#         li_review = i.text.split('\n')[9:]  # Skip the first 9 lines
#         li_review = [line for line in li_review if line not in ['\ue8dc', '\ue80d']]
        
#         if li_review:
#             li_review = li_review[:-2]  # Remove the last two items if the list is not empty
#         else:
#             li_review.append("BLANK")  # Add "BLANK" if the review is empty

#         # To remove unnecessary words - Translated by Google
#         li_review = [i.split('Translated by Google')[0].strip() if "Translated by Google" in i else i for i in li_review]
        
#         # Join the cleaned review list into a single string
#         user_comment = ' '.join(li_review)
#         user_comments.append(user_comment)

#         # Print the cleaned review
#         print(user_comment)
#         print("\n")

#     # Create a DataFrame and save to Excel
#     data = {
#         "Shop_Name": place.split(",")[0],
#         'Name': name_,
#         "Time_Of_Comment": time_,
#         "Rating_Stars": Rating_Stars,
#         "Review_Comment": user_comments
#     }
    
#     df = pd.DataFrame(data)
#     df.to_excel(f'{place.split(",")[0]}.xlsx', index=False)
    
# # List of places to scrape
# places = [
#     'Vape & Smoke Shop - Biscayne,2895 Biscayne Blvd, Miami, FL 33137, United States 25.8040176 -80.1917091',
#     'Fuego Life Vape Shop,401 Biscayne Blvd T5585, Miami, FL 33132, United States 25.8040939 -80.2303342',
#     'Caeli Smoke & Vape,16 SW 8th St, Miami, FL 33130, United States 25.7828172 -80.2301061',
#     # Add other places here...
# ]

# # Execute search place function for each place using multithreading
# with ThreadPoolExecutor(max_workers=5) as executor:
#     executor.map(searchplace, places)

# driver.quit()


NEW I came here today and got a great deal on a bunch of vapes! The guy that works there is so cool and nice! This will be my go to smoke shop here in miami ☺️


Super friendly cashier and very knowledgeable. Can't beat the prices either in the area


Horrible service, the boy at the counter has no idea what he is doing


Wilson.........................! Was awesome 👍😎 …


My dad comes here sometimes to get the crate on. They have red crate them and blue crate them and it’s in a powder they have a chocolate, or they have capsule form and they have all kinds of smoking paraphernalia made a smoke be with you says dark Vader, tobacco, flavored tobacco, chewing, tobacco belt, kids, and everything in +15 Like Share Response from the owner 4 months ago Hey Angeleeka,


BLANK


N/a


I’ve been coming here for years for all my recreational needs 😉. Today’s experience with Omar was a true delight!!!  He was kind, knowledge, patience and friendly! As always the store is very clean, extremely wel

In [ ]:
# chromedriver_path = "C:/Users/miraj/OneDrive/Desktop/projectMapsData-main/chromedriver.exe"
# service = Service(chromedriver_path)
# driver = webdriver.Chrome(service=service)

# driver.get("https://www.google.com/maps/@32.8191173,-97.3554595,10z?hl=en&entry=ttu")
# sleep(1)



# def searchplace():
#     # Locate the search box using its id
#     place = driver.find_element(By.ID, "searchboxinput")
#     locc = 'Vape & Smoke Shop - Biscayne,2895 Biscayne Blvd, Miami, FL 33137, United States 25.8040176 -80.1917091'
#     place.send_keys(locc)
#     sleep(2)  # Added sleep to ensure the input is fully typed
#     # Locate the search button using its id
#     submit = driver.find_element(By.ID, "searchbox-searchbutton")
#     submit.click()

#     sleep(4)


#     try:
#         shop = driver.find_element(By.CLASS_NAME, "hfpxzc")
#         shop.click()
#         sleep(2)

#     except:
#         pass


#     revieew_nav = driver.find_element(By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div:nth-child(3) > div > div > button:nth-child(2)')
#     revieew_nav.click()
#     sleep(3)

#     sort = driver.find_element(By.CSS_SELECTOR, 'button[aria-label="Sort reviews"]')
#     sort.click()
#     sleep(3)

#     new= driver.find_element(By.CSS_SELECTOR, '#action-menu > div:nth-child(2)')
#     new.click() 
#     sleep(4)


#    # Scroll to load more content

#     scrollable_div = driver.find_element(By.CSS_SELECTOR, "#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.XiKgde" )
#     last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)

#     scroll_count = 1
#     max_scrolls = 5

#     while scroll_count < max_scrolls:
#         driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
#         sleep(2)
#         new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
#         if new_height == last_height:
#             break
#         last_height = new_height
#         scroll_count += 1



#     review_boxs = driver.find_elements(By.CLASS_NAME, "jJc9Ad")

#     # Initialize lists to store names, times, and user comments
#     name_ = []
#     time_ = []
#     user_comments = []
#     Rating_Stars = []  

#     for i in review_boxs:
#         # Extract and store the name
#         name = i.find_element(By.CLASS_NAME, "d4r55")
#         name_.append(name.text)
        
#         # Extract and store the time ago
#         time_ago = i.find_element(By.CLASS_NAME, "rsqaWe")
#         time_.append(time_ago.text)

#         ratings = i.find_element(By.CLASS_NAME, "kvMYJc")
#         aria_label_value = ratings.get_attribute("aria-label")
#         Rating_Stars.append(aria_label_value)
#         # Extract and clean the review text
        
#         li_review = i.text.split('\n')[9:]  # Skip the first 9 lines
#         li_review = [line for line in li_review if line not in ['\ue8dc', '\ue80d']]
        
#         if li_review:
#             li_review = li_review[:-2]  # Remove the last two items if the list is not empty
#         else:
#             li_review.append("BLANK")  # Add "BLANK" if the review is empty

#         # To remove unnecessary words - Translated by Google
#         li_review=[i.split('Translated by Google')[0].strip() if "Translated by Google" in i else i for i in li_review]
        
#         # Join the cleaned review list into a single string
#         user_comment = ' '.join(li_review)
#         user_comments.append(user_comment)

#          # Print the cleaned review
#         print(user_comment)
#         print("\n")


#     dict={
        
#         "Shop_Name":locc.split(",")[0],
#         'Name': name_,
#         "Time_Of_Comment": time_,
#         "Rating_Stars": Rating_Stars,
#         "Review_Comment": user_comments
#     }
    
#     df = pd.DataFrame(dict)
#     df.to_excel('Caeli Smoke & Vape.xlsx', index=False)
    
# # Execute the search place function
# searchplace()
# sleep(2)



In [32]:
# latitude=[]
# longitude=[]
# shop=[]
# def findshopss():
   
#             # Click the "nearby" button
#             findshop = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[4]/div[3]/button')
#             findshop.click()
#             sleep(5)  # Wait for the menu to open

#             # Enter "vape" in the search field
#             place = driver.find_element(By.ID, "searchboxinput")
#             kw = "vape"
#             place.send_keys(kw)
#             sleep(1)

#             # Click the search button
#             submit = driver.find_element(By.ID, "searchbox-searchbutton")
#             submit.click()
#             sleep(5)  # Wait for the search results to load

#             All_shops = driver.find_elements(By.CLASS_NAME, "Nv2PK")
            
#             print(f"Number of shops found: {len(All_shops)}")  

#             for i in All_shops:
                    
#                 i.click()
#                 sleep(2)
                
#                 dir = driver.find_element(By.CSS_SELECTOR, 'button[data-value="Directions"]')
#                 dir.click()
#                 sleep(2)

#                 current_url = driver.current_url
                
#                 parsed_url = urlparse(current_url)

#                 # Extract the path and split by '/'
#                 path = parsed_url.path.split('/')

#                 # Find the part that contains the coordinates after the '@' symbol
#                 coordinates = [part for part in path if '@' in part]

#                 if coordinates:
#                     # Extract the latitude and longitude
#                     lat_lng = coordinates[0].split('@')[1].split(',')[0:2]
                    
#                     latitude.append(lat_lng[0])
#                     longitude.append(lat_lng[1])

#                 else:
#                       latitude.append("N/A")
#                       longitude.append("N/A")
                
#                 dir = driver.find_element(By.CSS_SELECTOR, 'button[data-tooltip="Close directions"]')
#                 dir.click()
#                 sleep(2)

#                 All_shop = driver.find_elements(By.CLASS_NAME, "Nv2PK")


In [2]:
import pandas as pd
import glob
 
# Specify the path where your CSV files are located
path = 'output'
 
# Use glob to get a list of all CSV files in the specified path
all_files = glob.glob(path + "*.xlsx")
 
# List to hold all the DataFrames
df_list = []
 
# Loop through each file and read it into a DataFrame
for file in all_files:
    df = pd.read_csv(file)
    df_list.append(df)
 
# Concatenate all DataFrames vertically
merged_df = pd.concat(df_list, axis=0)
 
# Remove duplicate rows
final_df = merged_df.drop_duplicates()
 
# Save the final DataFrame to a new CSV file
final_df.to_csv('merged_output.csv', index=False)

ValueError: No objects to concatenate

In [4]:
import pandas as pd

df1 = pd.read_excel("output/scrape_515 palm beach Road, south bay, Florida 33493_gas station near 515 palm beach Road, south bay, Florida 33493_2024-07-29_22-21-42.xlsx")
df2 = pd.read_excel("output/scrape_515 palm beach Road, south bay, Florida 33493_liquor store_2024-07-29_22-07-30.xlsx")
df3 = pd.read_excel("output/scrape_515 palm beach Road, south bay, Florida 33493_smoke shop near 515 palm beach Road, south bay, Florida 33493_2024-07-29_22-14-25.xlsx")

In [5]:

final_df = pd.concat([df1, df2, df3], ignore_index=True)

In [6]:
final_df

,shop,rating,total_ratings,type,address,website,number,latitude,longitude
0,Marathon Gas,4.0,(30),Gas station,"890 US Hwy 27, South Bay, FL 33493, United States",https://www.marathonbrand.com/Stations/Station...,+1 561-996-6599,26.669876,-80.729253
1,Marathon,1.0,(1),Gas station,"105 US Hwy 27, South Bay, FL 33493, United States",NaN,+1 561-992-8207,26.669914,-80.729253
2,G R Gas Station,4.0,(95),Gas station,"105 US Hwy 27, South Bay, FL 33493, United States",NaN,+1 561-996-6599,26.663620,-80.718083
3,Chevron,1.8,(36),Gas station,"19345 FL-7, Boca Raton, FL 33498, United States",https://www.chevronwithtechron.com/,+1 561-451-1166,26.388549,-80.206950
4,Sunoco Gas Station,3.9,(37),Gas station,"2490 PGA Boulevard, Palm Beach Gardens, FL 334...",https://www.sunoco.com/locations/store/2490-pg...,+1 561-627-4570,26.844264,-80.074322
...,...,...,...,...,...,...,...,...,...
78,World of Smoke & Vape - Homestead,4.9,(77),Smoke shop,"1631 NE 8th St, Homestead, FL 33033, United St...",https://worldofsmokenvape.com/?utm_source=gmb&...,+1 786-404-3860,25.749944,-80.329115
79,Milano smoke shop,4.8,(97),Tobacco shop,"30318 Old Dixie Hwy, Homestead, FL 33033, Unit...",NaN,+1 786-878-9261,25.486416,-80.464657
80,1st smoke shop and Cbd dispensary,4.9,(61),Cannabis store,"2194 Main St Suite B, Dunedin, FL 34698, Unite...",https://www.disposabalevape.com/,+1 727-953-3389,28.019742,-82.751007
81,Vape & Smoke Shop - Biscayne,4.8,"(1,006)",Vaporizer store,"2895 Biscayne Blvd, Miami, FL 33137, United St...",http://www.vapensmokeshop.com/portfolio-view/b...,+1 786-703-7692,25.886165,-80.249713


In [7]:
final_df=final_df.drop_duplicates()

In [8]:
final_df

,shop,rating,total_ratings,type,address,website,number,latitude,longitude
0,Marathon Gas,4.0,(30),Gas station,"890 US Hwy 27, South Bay, FL 33493, United States",https://www.marathonbrand.com/Stations/Station...,+1 561-996-6599,26.669876,-80.729253
1,Marathon,1.0,(1),Gas station,"105 US Hwy 27, South Bay, FL 33493, United States",NaN,+1 561-992-8207,26.669914,-80.729253
2,G R Gas Station,4.0,(95),Gas station,"105 US Hwy 27, South Bay, FL 33493, United States",NaN,+1 561-996-6599,26.663620,-80.718083
3,Chevron,1.8,(36),Gas station,"19345 FL-7, Boca Raton, FL 33498, United States",https://www.chevronwithtechron.com/,+1 561-451-1166,26.388549,-80.206950
4,Sunoco Gas Station,3.9,(37),Gas station,"2490 PGA Boulevard, Palm Beach Gardens, FL 334...",https://www.sunoco.com/locations/store/2490-pg...,+1 561-627-4570,26.844264,-80.074322
...,...,...,...,...,...,...,...,...,...
78,World of Smoke & Vape - Homestead,4.9,(77),Smoke shop,"1631 NE 8th St, Homestead, FL 33033, United St...",https://worldofsmokenvape.com/?utm_source=gmb&...,+1 786-404-3860,25.749944,-80.329115
79,Milano smoke shop,4.8,(97),Tobacco shop,"30318 Old Dixie Hwy, Homestead, FL 33033, Unit...",NaN,+1 786-878-9261,25.486416,-80.464657
80,1st smoke shop and Cbd dispensary,4.9,(61),Cannabis store,"2194 Main St Suite B, Dunedin, FL 34698, Unite...",https://www.disposabalevape.com/,+1 727-953-3389,28.019742,-82.751007
81,Vape & Smoke Shop - Biscayne,4.8,"(1,006)",Vaporizer store,"2895 Biscayne Blvd, Miami, FL 33137, United St...",http://www.vapensmokeshop.com/portfolio-view/b...,+1 786-703-7692,25.886165,-80.249713


In [9]:
final_df.to_excel("final_output.xlsx", index=False)

In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time


driver = webdriver.Chrome(service=service)

websites = [
    'https://nepa2wholesale.com/',
    # Add all 100 websites here
]

for website in websites:
    try:
        driver.get(website)
        time.sleep(3)  # Wait for the page to load (adjust as needed)
        
        # Handle age verification
        try:
            # Example: Click an "Enter" button for age verification
            enter_button = driver.find_element(By.XPATH, '//button[contains(text(), "Enter")]')
            enter_button.click()
            time.sleep(3)  # Wait for the page to load after age verification
        except Exception as e:
            print(f'No age verification needed or failed to bypass for {website}: {e}')
        
        # Extract data from the landing page
        page_title = driver.title
        print(f'Website: {website}, Title: {page_title}')
        
        # Extract main content while ignoring header and footer
        try:
            # Assuming main content is within a specific element, e.g., <div id="main-content">
            main_content = driver.find_element(By.CSS_SELECTOR, 'body')
            header = main_content.find_element(By.TAG_NAME, 'header')
            footer = main_content.find_element(By.TAG_NAME, 'footer')
            
            # Remove header and footer content
            driver.execute_script("""
                var header = arguments[0];
                var footer = arguments[1];
                if (header) { header.parentNode.removeChild(header); }
                if (footer) { footer.parentNode.removeChild(footer); }
            """, header, footer)
            
            # Get the remaining content
            content_text = main_content.text
            print(f'Content: {content_text}')  # Print first 100 characters of content
            
        except Exception as e:
            print(f'Error extracting main content for {website}: {e}')
        
    except Exception as e:
        print(f'Error with {website}: {e}')

driver.quit()


Website: https://nepa2wholesale.com/, Title: Nepa Wholesale – A Complete Store For Products
Content: Explore Our Brands
New to NEPA Wholesale? Fill in and we will contact you.
Business Name
Email
Phone
HQD Cuvie X
HQD Cuvie Bar
HQD Cuvie Plus
HQD Mars
HQD Cuvie Glaze
HQD Cuvie Slick
SHOP NOW
HQD Cuvie X 5% Grape 1X5Pk Disposable (25000) (6937105490921)
Login | Register for price & stock
HQD Cuvie X 5% Lush Ice 1X5Pk Disposable (25000) (6937105490969)
Login | Register for price & stock
Fume Eternity Disposable
Fume Extra Disposable Vape
Fume Frutia Disposable
Fume Recharge Disposable
Fume Ultra Disposable
Fume Infinity Disposable
We Fume Disposable
SHOP MORE
We Fume 5Pk Clear 5% (30000) (6941821728039)
Login | Register for price & stock
We Fume 5Pk Strawberry Kiwi 5% (30000) (6941821727957)
Login | Register for price & stock
Cigarettes
Cigars
Rolling Papers & Cones
Wraps
Tobaccos
Smoking Accessories
MORE CATEGORIES
Throwback Leaf 8-5Ct Sunset (860009329904)
Login | Register for price & 